### Metric 1: Average commutator norm

In [ ]:
import json
import numpy as np
from ansatz import Ansatz
import pennylane as qml
from twirler.symmetry_groups import create_subgroup_from_permutations, create_symmetric_group, create_induced_subgroup_from_permutations
from twirler.generators import get_ansatz_generators
from twirler.induced_representation import derive_unitaries_angle_embedding_analytic
from joblib import Parallel, delayed

results = {}
n_qubits = 7

S = create_symmetric_group(n_qubits)

with open(f"groups/subgroups_{n_qubits}.json", "r") as f:
    subgroups = json.load(f)

subgroup_unitaries = {}
for k, groups in subgroups.items():
    subgroup_unitaries[k] = []
    print("Processing subgroup:", k)
    for generators in groups:
        new_generators = []
        for g in generators:
            new_generators.append(tuple(g))

        K = create_subgroup_from_permutations(S, new_generators)
        unitaries = derive_unitaries_angle_embedding_analytic(K)
        subgroup_unitaries[k].append(unitaries)

results[n_qubits] = {}
for depth in range(1, 6):
    print("Depth:", depth)
    results[n_qubits][depth] = {}
    for ansatz_id in range(1, 20):
        print("  Ansatz ID:", ansatz_id)
        results[n_qubits][depth][ansatz_id] = {}
        super_ansatz = Ansatz(ansatz_id, n_qubits, depth)
        ansatz = super_ansatz.get_ansatz()
        generators = get_ansatz_generators(ansatz)

        # Precompute once per ansatz
        G_full_list = [
            qml.matrix(qml.Hermitian(gen_observable, wires=wires), wire_order=range(n_qubits))
            for (gen_observable, wires, _, _, _) in generators
        ]

        def _accumulate_for_unitary_dict(unitary_dict):
            m = len(unitary_dict)
            local_total = 0.0
            for U_s in unitary_dict.values():
                for G_full in G_full_list:
                    commuter = U_s @ G_full - G_full @ U_s
                    local_total += np.linalg.norm(commuter, ord='fro')
            return local_total, m * len(G_full_list)

        for k in subgroup_unitaries:
            partials = Parallel(n_jobs=-1, backend="threading")(
                delayed(_accumulate_for_unitary_dict)(unitary) for unitary in subgroup_unitaries[k]
                )
            total_norm = sum(x for x, _ in partials)
            total_count = sum(c for _, c in partials)
            avg_norm = total_norm / total_count
            results[n_qubits][depth][ansatz_id][k] = avg_norm

with open(f"results/results_average_commutator_norm_{n_qubits}.json", "w") as f:
    json.dump(results, f, indent=4)

Processing subgroup: 1


100%|██████████| 1/1 [00:00<00:00, 499.86it/s]


Processing subgroup: 2


100%|██████████| 30/30 [00:00<00:00, 618.41it/s]


Processing subgroup: 3


...

Processing subgroup: 2520


100%|██████████| 1/1 [00:00<00:00,  1.06it/s]


Processing subgroup: 5040


100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


Depth: 1
  Ansatz ID: 1
  Ansatz ID: 2
  Ansatz ID: 3
  Ansatz ID: 4
  Ansatz ID: 5
  Ansatz ID: 6
  Ansatz ID: 7
  Ansatz ID: 8
  Ansatz ID: 9
  Ansatz ID: 10
  Ansatz ID: 11
  Ansatz ID: 12
  Ansatz ID: 13
  Ansatz ID: 14
  Ansatz ID: 15
  Ansatz ID: 16
  Ansatz ID: 17
  Ansatz ID: 18
  Ansatz ID: 19
Depth: 2
  Ansatz ID: 1
  Ansatz ID: 2
  Ansatz ID: 3
  Ansatz ID: 4
  Ansatz ID: 5
  Ansatz ID: 6
  Ansatz ID: 7
  Ansatz ID: 8
  Ansatz ID: 9
  Ansatz ID: 10
  Ansatz ID: 11
  Ansatz ID: 12
  Ansatz ID: 13
  Ansatz ID: 14
  Ansatz ID: 15
  Ansatz ID: 16
  Ansatz ID: 17
  Ansatz ID: 18
  Ansatz ID: 19
Depth: 3
  Ansatz ID: 1
  Ansatz ID: 2
  Ansatz ID: 3
  Ansatz ID: 4
  Ansatz ID: 5
  Ansatz ID: 6
  Ansatz ID: 7
  Ansatz ID: 8
